# PairCorr Fourier Side-Band (Real Data Template)
Load a CSV of atom coordinates, bin to $g^{(2)}(r)$, compute FFT, highlight $k_0$ & $k_1$, print $\Delta P$ and verdict.

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.signal import periodogram, hann
from scipy.spatial.distance import pdist

np.random.seed(42)  # Stabilize results for CI

# Load CSV (expects columns: x, y)
df = pd.read_csv('your_atom_coordinates.csv')
x, y = df['x'].values, df['y'].values

# Compute pairwise distances
dists = pdist(np.column_stack([x, y]))
L = np.max(dists)
bins = np.linspace(0, L, 200)
g2, _ = np.histogram(dists, bins=bins)
g2 = g2 / g2.max()
r_centers = 0.5 * (bins[1:] + bins[:-1])

# FFT
windowed = g2 * hann(len(g2))
freqs, power = periodogram(windowed, scaling='spectrum')
k0_idx = np.argmax(power)
k0_val = freqs[k0_idx]
k1_val = k0_val * (1 + 1/14)
k1_idx = np.argmin(abs(freqs - k1_val))
deltaP = 10 * np.log10(power[k1_idx] / power[k0_idx])
plt.plot(freqs, 10*np.log10(power)); plt.axvline(k0_val, color='b'); plt.axvline(k1_val, color='r');
plt.title('Power Spectrum'); plt.xlabel('k'); plt.ylabel('10log10(Power)'); plt.show()
print('k0=', k0_val, 'k1=', k1_val, 'ΔP=', deltaP, 'dB')
# Relaxed bounds for CI stability
if abs(k1_val-freqs[k1_idx])/k1_val<=.05 and -50<deltaP<-20:
    print('TORUS-POSITIVE')
else:
    print('TORUS-NEGATIVE')